# Análisis y preprocesado del dataset Severance
### Importación de librerías

In [7]:
import os
import cv2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

grosor_borde = 4  # Establece el grosor en píxeles del borde blanco a eliminar

In [2]:
'''
This function receives a string with the filename of the image to read,
and a flag indicating if we want to read it in color/RGB (flagColor=1) or gray level (flagColor=0)

Example of use:
im1=readIm(get_image('apple.jpg'),0)
'''


def readIm(filename, flagColor=1):
    # cv2 reads BGR format
    im = cv2.imread(filename)
    # change to  RGB and return the image
    if (flagColor):
        return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    # change from BGR to grayscale instead if flag is 0
    return cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)


'''
This function receives an array of arbitrary real numbers (that could include even negative values),
and returns an 'image' in the range [0,1].
flag_GLOBAL allows the user to normalize the whole image (including all channels) or to normalize
each channel/band independently.
'''


def rangeDisplay01(im, flag_GLOBAL=True):
    im = im.astype(float)
    if flag_GLOBAL:
        im = (im - im.min()) / (im.max() - im.min())
    else:
        # bands normalization
        for band in range(im.shape[2]):
            im[:, :, band] = (im[:, :, band] - im[:, :, band].min()) / (im[:, :, band].max() - im[:, :, band].min())
            # Note: remember that, for plt.imshow with RGB data, the valid range is [0..1] for floats and [0..255] for integers.
    return im


"""
Función para mostrar imágenes en pantalla en color y blanco negro. Permite realizar
aumento sobre las mismas para apreciar un mayor detalle.

Entrada:
    im: imagen leída en formato ndarray
    title: nombre que recibe el marco en pantalla
    factor: factor de aumento de la image, "zoom"
"""


def displayIm(im, title='Result', factor=2):
    # First normalize range
    max = np.max(im)
    min = np.min(im)
    if min < 0 or max > 255:
        im = rangeDisplay01(im, flag_GLOBAL=True)
    if len(im.shape) == 3:
        # im es tribanda
        plt.imshow(im, cmap='jet')
    else:
        # im es monobanda
        plt.imshow(im, cmap='gray')
    figure_size = plt.gcf().get_size_inches()
    plt.gcf().set_size_inches(factor * figure_size)
    plt.title(title)
    plt.xticks([]), plt.yticks([])  # eliminamos numeración
    plt.show()

### Lectura de las imágenes

In [3]:
img_path = "../"
print(os.getcwd())
# XML sin preprocesar
os.chdir(img_path)

trainData = pd.read_excel('SeveranceA.xls', header=None)

# .csv procesado
#filas = get_thumbnails(trainData, img_path)
#train = pd.DataFrame(filas, columns=trainData.columns)
display(trainData)
os.chdir("SeveranceA_Thumbnails/")

C:\Users\Cris1\Desktop\TFG\datasets\Severance\SeveranceA_Thumbnails


,0,1,2,3,4,5,6
0,actinickeratosis,60,M,R/O AK\nR/O SCC,actinickeratosis;squamouscellcarcinoma;,Lt. cheek,Head&Neck
1,actinickeratosis,68,F,R/O AK,actinickeratosis;,perioral area,Head&Neck
2,actinickeratosis,39,F,R/O AK\nR/O SK,actinickeratosis;seborrheickeratosis;,nose,Head&Neck
3,actinickeratosis,63,F,R/O AK,actinickeratosis;,cheek,Head&Neck
4,actinickeratosis,66,F,R/O SK\nR/O AK,seborrheickeratosis;actinickeratosis;,nose,Head&Neck
...,...,...,...,...,...,...,...
10421,spitznevus,18,F,R/O inflammed intradermal nevus,melanocyticnevus;,cheek,Head&Neck
10422,spitznevus,13,F,R/O GP,pyogenicgranuloma;,Lt. calf,Leg
10423,spitznevus,14,F,R/O spitz nevus,otherbenign;,lower leg,Leg
10424,spitznevus,8,M,R/O hemangioma,hemangioma;,Rt. thigh,Leg


In [4]:
print(os.getcwd())

C:\Users\Cris1\Desktop\TFG\datasets\Severance\SeveranceA_Thumbnails


### Tratamiento y división

Se leen de disco las imágenes de gran tamaño y se dividen en miniaturas para su tratamiento. Se procede a la eliminación de posibles bordes blancos restantes mediante la eliminación de 4 píxeles de margen.

In [5]:
import pathlib

# Creamos directorio de imágenes recortadas
if not os.path.exists('thumbnails'):
    os.makedirs('thumbnails')

# Listamos las imágenes en rejilla disponibles
files = [f for f in pathlib.Path().iterdir() if f.is_file()][:-2]

names = []
diss_class = []
j = -1

trainProcessed = []

for f in files:
    # Retiramos la extensión del fichero
    name = str(f)[:-4]

    # Leemos la imagen
    image = cv2.imread(str(f), cv2.IMREAD_UNCHANGED)[2:-6]

    # Dividimos la rejilla
    image_split = np.split(image[1:-1], 8)

    i = 0
    for im in image_split:
        # Recorta la imagen en columnas
        image_split_col = np.split(im[:, 2:-3], 15, axis=1)
        # Para cada imagen
        for imcol in image_split_col:
            # Eliminar el borde
            img_sin_borde = imcol[grosor_borde:-grosor_borde, grosor_borde:-grosor_borde]
            if (np.count_nonzero(imcol == 255) > 20000):  # Si es imagen de separador con nombre
                j += 1  # Cambiamos de etiqueta al siguiente paciente
                continue
            else:
                actual = list(trainData.iloc[j])
                actual.append(f"{name}_{i}.png")
                trainProcessed.append(actual)
            # Guardamos la imagen y añadimos su nombre
            cv2.imwrite(f"thumbnails/{name}_{i}.png", img_sin_borde)
            names.append(f"{name}_{i}.png")
            i += 1

        diss_class.append(name)

Ahora, podemos guardar todas las imágenes en disco creando un nuevo fichero csv que permita fácilmente identificar los metadatos de cada miniatura

In [6]:
# Creamos el dataframe con las imágenes recortadas
trainProcessedDF = pd.DataFrame(trainProcessed)
trainProcessedDF.columns = ["label", "age", "sex", "abrev_diagnosis", "diagnosis", "fine_loc", "loc", "image"]

# Salvamos a disco duro la información
display(trainProcessedDF)
trainProcessedDF.to_csv("severanceTrainingSet_thumbnails.csv", sep=',', encoding='utf-8')

,label,age,sex,abrev_diagnosis,diagnosis,fine_loc,loc,image
0,actinickeratosis,60,M,R/O AK\nR/O SCC,actinickeratosis;squamouscellcarcinoma;,Lt. cheek,Head&Neck,001_0.png
1,actinickeratosis,60,M,R/O AK\nR/O SCC,actinickeratosis;squamouscellcarcinoma;,Lt. cheek,Head&Neck,001_1.png
2,actinickeratosis,60,M,R/O AK\nR/O SCC,actinickeratosis;squamouscellcarcinoma;,Lt. cheek,Head&Neck,001_2.png
3,actinickeratosis,60,M,R/O AK\nR/O SCC,actinickeratosis;squamouscellcarcinoma;,Lt. cheek,Head&Neck,001_3.png
4,actinickeratosis,68,F,R/O AK,actinickeratosis;,perioral area,Head&Neck,001_4.png
...,...,...,...,...,...,...,...,...
35165,spitznevus,8,F,R/O Spitz nevus,otherbenign;,abdomen,Trunk,380_88.png
35166,spitznevus,8,F,R/O Spitz nevus,otherbenign;,abdomen,Trunk,380_89.png
35167,spitznevus,8,F,R/O Spitz nevus,otherbenign;,abdomen,Trunk,380_90.png
35168,spitznevus,8,F,R/O Spitz nevus,otherbenign;,abdomen,Trunk,380_91.png
